In [1]:
import cv2

# Load pre-trained car detection classifier (Haar Cascade)
car_cascade = cv2.CascadeClassifier(r"C:\Users\quqio\anaconda3\Lib\site-packages\cv2\data\cars.xml")

# Open the video file
video_path = r"C:\Users\quqio\OneDrive\Desktop\test.mp4.mp4"
cap = cv2.VideoCapture(video_path)

In [2]:
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate video duration(seconds)
duration = frame_count / fps

print(f"Vedio duration：{duration:.2f} seconds")

Vedio duration：11.18 seconds


In [3]:
car_count = 0
previous_detections = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect cars in the frame
    cars = car_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # List to store IDs of newly detected cars
    new_cars = []

    # Loop through each detected car
    for (x, y, w, h) in cars:
        # Check if the car is a new detection
        is_new_detection = True
        for (prev_x, prev_y, prev_w, prev_h) in previous_detections:
            if (x >= prev_x and x <= prev_x + prev_w) or (prev_x >= x and prev_x <= x + w):
                is_new_detection = False
                break

        # If it's a new detection, increment car count and store the detection
        if is_new_detection:
            new_cars.append((x, y, w, h))
            car_count += 1
            previous_detections.append((x, y, w, h))

    # Draw bounding boxes around newly detected cars
    for (x, y, w, h) in new_cars:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the resulting frame with bounding boxes and car count
    cv2.putText(frame, f'Car Count: {car_count}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print(f"amount of the cars：{car_count}")

amount of the cars：10


In [4]:
class Traffic:
    def __init__(self, cars, time):
        self.cars = cars
        self.time = time

class Analyze_traffic(Traffic):
    def __init__(self, cars, time):
        super().__init__(cars, time)
        self.result = self.calculate_traffic_density()

    def calculate_traffic_density(self):
        if self.time > 0:
            return self.cars / self.time
        else:
            return 0  # Avoid division by zero error if time is zero

    def is_rush_hour(self):
        if self.result > 1:  
            print("Rush hour!")
        else:
            print("Normal Traffic")


if __name__ == '__main__':
    # Create Analyze_traffic object with cars and time values
    cars_count = float(car_count)
    time_interval = float(duration)

    analyzer = Analyze_traffic(cars_count, time_interval)

    # Calculate traffic density and check if it's rush hour
    print(f"Traffic density: {analyzer.result} cars per seconds")
    analyzer.is_rush_hour()

Traffic density: 0.894627760299402 cars per seconds
Normal Traffic
